<a href="https://colab.research.google.com/github/yuweiweiwei/yuweiweiwei/blob/main/Copy_of_Hwk2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preliminaries

In [ ]:
# Import data files from CoLab (if using colab if not skip this step, but make sure the data and this file are int he same directory.
from google.colab import drive
drive.mount('/content/drive/')


In [ ]:
cd $REPLACE_WITH_YOUR_PATH$

The next thing we need to do is import all the modules and libraries that we need. Don't worry about this for now, we will discuss this later.

If you run into errors saying "module not found" or something like that, it is likely you haven't installed it. You should google how to use "pip install" or "conda install" to download packages that you need. The

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
import astropy.units as u
from matplotlib.colors import LogNorm, PowerNorm, FuncNorm
import scipy.interpolate as interp

# Problem 1

First we are going to read in a fits file. This file was taken at the Gemini North Telescope using the GMOS-N instrument. This instrument is what is known as an imaging spectrograph.

In [ ]:
$YOUR_FILE_NAME$ = "N20240704S0106_image.fits"  # Change this to your file
hdul = fits.open($File_name_here$)
image_data = hdul[1].data  # Assuming the image is in the primary HDU
hdul.close()

FileNotFoundError: [Errno 2] No such file or directory: 'N20240704S0106_image.fits'

First, display the data. You can do it using the code below OR open it up using JS9. What do you notice?

In [ ]:
def sinh_norm(image_data, lower_percentile=1, upper_percentile=99):
    """
    Creates a sinh-based normalization function for image scaling.

    Parameters:
    - image_data (ndarray): 2D array representing the image.
    - lower_percentile (float): Lower percentile for contrast scaling (default: 1).
    - upper_percentile (float): Upper percentile for contrast scaling (default: 99).

    Returns:
    - FuncNorm: Normalization function to apply to images.
    """
    # Compute vmin and vmax using percentiles
    vmin, vmax = np.percentile(image_data, [lower_percentile, upper_percentile])

    # Define sinh scaling functions
    def sinh_forward(x):
        return np.sinh((x - vmin) / (vmax - vmin))

    def sinh_inverse(y):
        return vmin + (vmax - vmin) * np.arcsinh(y)

    # Return the normalization object
    return FuncNorm((sinh_forward, sinh_inverse), vmin=vmin, vmax=vmax)

In [ ]:
# Assign the normalization to norm
norm = sinh_norm(image_data)

In [ ]:
plt.imshow(image_data, origin='lower', cmap='gray', norm=norm)
plt.colorbar()
plt.xlabel('x pix')
plt.ylabel('y pix')
plt.show()

*Insert your notes here*

Now, we are going to open the headers and take a look. This image has two headers in particular. The first one is the primary header for the image, and the second is a result from the reduction process. Open them up and have a look. Remember that indexing in python starts at 0, so hdul[0] is the first header. Assign variables and open them up

In [ ]:
$first_header_name$ = hdul[0].header
$name_of_second_header$ = hdul[1].header

In [ ]:
# print the two headers
$first_header_name$

In [ ]:
$second_header_name$

What kind of data type is the header? (It is not “technically” this type, but it’s close enough)

Hint: There is a function you've seen that can tell you this

*Your answer here*

What data type is the image data?

What is the "target" of the image? i.e. what was the astronomical object this image was pointed at? Don't just report the result in the header as it is just a name, do some investigating to figure out what specifically this object is.

In [ ]:
print($your_first_header_name$['OBJECT'])

*Your answer here*

Next, run the following cell to crop to the center of the image. This is where the object is.

In [ ]:
def crop_by_pixels(ffi, x_center, y_center):
  # Define crop parameters
  crop_size = 200  # Half of the width/height of the cutout

  # Define pixel range for cropping
  x_min, x_max = x_center - crop_size, x_center + crop_size
  y_min, y_max = y_center - crop_size, y_center + crop_size

  # Crop the image
  cropped_data = ffi[y_min:y_max, x_min:x_max]

  # Plot cropped image
  plt.figure(figsize=(6, 6))
  plt.imshow(cropped_data, origin='lower', cmap='gray', norm=norm)
  plt.colorbar(label="Flux")
  plt.title(f"Cropped Image Centered at ({x_center}, {y_center})")
  plt.show()

  return cropped_data

In [ ]:
cropped_image = crop_by_pixels($your_image_data$, 1609, 1044)

This image is okay, but We should rescale it to get it better looking. Change the percentile values below to find one that you like and run the function that compares both of them side by side.

In [ ]:
new_norm = sinh_norm(cropped_image, lower_percentile=$Your_value_here$, upper_percentile=$Your_value_here$)

In [ ]:
def plot_both():
  plt.figure(figsize=(12, 6))

  plt.subplot(1, 2, 1)
  plt.imshow(cropped_image, origin='lower', cmap='gray', norm=norm)
  plt.colorbar()
  plt.xlabel('x pix')
  plt.ylabel('y pix')
  plt.title('Original Normalization')

  plt.subplot(1, 2, 2)
  plt.title('New Normalization')
  plt.imshow(cropped_image, origin='lower', cmap='gray', norm=new_norm)
  plt.colorbar()
  plt.xlabel('x pix')
  plt.ylabel('y pix')
  plt.show()

plot_both()

What do you notice between these two images?

*Your answer here*

# Problem 2

Next we are going to take a look at spectral data. This data set a csv table of solar values taken from https://www.nrel.gov/grid/solar-resource/spectra-astm-e490.html. Read in the csv file and run the given function to plot it.

In [ ]:
# Read in solar spectrum using np.gen from text. Specifiy the delimiter as a comma (make sure it's a string), and set skip_header = 1
$name_of_Spectrum$ = ...

Use the python print function to print the values. Use the original csv file to determine which column is wavelength, and which is the irradiance (irradiance is the recived energy per unit time per unit wavelength, per unit area).

In [ ]:
print($name_of_Spectrum$)

assign a column to either wavelength or irradiance and comment what the units are

In [ ]:
wl = $name_of_Spectrum$[:, 0] # units?
irrad = $name_of_Spectrum$[:, 1] # units?

In [ ]:
# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(wl, irrad, linestyle='-', color='b')
plt.xscale('log')
plt.xlabel('Wavelength $Units?$')
plt.ylabel('Irradiance $units?')
plt.title('Solar Spectrum')
plt.xlim(min(wl),1e1) # limit between 0.1 and 1 micron
plt.grid(True)
plt.show()

Here is a "cropped" version of the spectra that spans from the optical into the near-IR. What are some things that you notice about the spectrum?

*your answer here*

There is an important thing to note here. These values are resentative at the top of the atmosphere. So, imagine we are trying to observe the sun in the optical regime. What kind of flux (irradiance) might we expect to our instrument? Let's find out

You might remember instruments use filters to limit the wavelengths that we observe. Read in the filter transmission file and let's discuss what is going on. This is the transmission profile from the g_high filter on VLT/FORS (https://www.eso.org/sci/facilities/paranal/instruments/fors/inst/Filters/curves.html).  The g high filter is a wide band optical filter

In [ ]:
filter_trans = np.genfromtxt('g_HIGH.txt', skip_header=2)

In [ ]:
filter_wl = filter_trans[:, $which_column_is_wl$]
filter_trans = filter_trans[:, $which_column_is_transmission$]

In [ ]:
# For some reason the values are all reversed so we must correct that
filter_wl = filter_wl[::-1]
filter_trans = filter_trans[::-1]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(filter_wl, filter_trans, linestyle='-', color='b')
plt.xlabel('Wavelength [nm]')
plt.ylabel('% Transmission')
plt.title('g high Filter Transmission')
plt.grid(True)
plt.show()

Okay, so we see that the transmission (or the percent of light that is let through) is somewhere between 375 and 575 nm. What we want to do is see how much light from the solar spectrum makes it past the filter.

A couple of technicalities here. This transmission plot is in percent transmission. The first thing we want to do is normalize it between 0 and 1. Do that below and replot to verify you did it correctly. The graph should look exactly the same except with transmission values between 0 and 1.

In [ ]:
# Normalize your values of transmission between 0 and 1. Remember filter transmission is an array
$Your_code_here$

In [ ]:
#Add your normalized array name where it says $normalized_array$
plt.figure(figsize=(10, 6))
plt.plot(filter_wl, $normalized_array$, linestyle='-', color='b')
plt.xlabel('Wavelength [nm]')
plt.ylabel('% Transmission')
plt.title('g high Filter Transmission')
plt.grid(True)
plt.show()

The next thing we need to take care of is the issue of matching wavelengths. You can either choose to convert the transmission wavelengths into microns to match the spectra or vice versa. We're going to choose that way because the resolution of the transmission data is higher and you will achieve more acurate results when interpolating the filter onto the spectra - this is the kind of decisions you will need to make yourself later on down the line as an astronomer (or data scientist in general).

In [ ]:
# First get the units to match (nm -> microns)
filter_wl = $convert_the_wavelengths_using_a_conversion_factor$

In [ ]:
# Replot
plt.figure(figsize=(10, 6))
plt.plot(filter_wl, filter_trans, linestyle='-', color='b')
plt.xlabel('Wavelength [microns]')
plt.ylabel('% Transmission')
plt.title('g high Filter Transmission')
plt.grid()


In [ ]:
# Find the minimum value of the transmission filter wavelengths and the spectrum wl
print(min(filter_wl))
print(min(wl))

Which should we use as the start for the matching region?

In [ ]:
# Set boundaries
min_wl = min($filter_wavelength_or_spectra_wl?) # microns
max_wl = 1  # microns

In [ ]:
# Check that values are expected
min_wl, max_wl

In [ ]:
# You can just run this.
def filter_wavelength_range(spectrum_wavelengths, spectrum_values,
                            filter_wavelengths, filter_values,
                            wavelength_min, wavelength_max):
    """
    Keeps only the wavelengths and values that fall within the specified range.

    Parameters:
    - spectrum_wavelengths (ndarray): Wavelengths of the spectrum.
    - spectrum_values (ndarray): Corresponding values (flux, intensity, etc.) of the spectrum.
    - filter_wavelengths (ndarray): Wavelengths of the transmission curve.
    - filter_values (ndarray): Corresponding transmission values.
    - wavelength_min (float): Minimum wavelength to keep.
    - wavelength_max (float): Maximum wavelength to keep.

    Returns:
    - filtered_spectrum_wavelengths (ndarray): Spectrum wavelengths within range.
    - filtered_spectrum_values (ndarray): Spectrum values within range.
    - filtered_filter_wavelengths (ndarray): Transmission wavelengths within range.
    - filtered_filter_values (ndarray): Transmission values within range.
    """

    # Mask spectrum data
    spectrum_mask = (spectrum_wavelengths >= wavelength_min) & (spectrum_wavelengths <= wavelength_max)
    filtered_spectrum_wavelengths = spectrum_wavelengths[spectrum_mask]
    filtered_spectrum_values = spectrum_values[spectrum_mask]

    # Mask filter data
    filter_mask = (filter_wavelengths >= wavelength_min) & (filter_wavelengths <= wavelength_max)
    filtered_filter_wavelengths = filter_wavelengths[filter_mask]
    filtered_filter_values = filter_values[filter_mask]

    return (filtered_spectrum_wavelengths, filtered_spectrum_values,
            filtered_filter_wavelengths, filtered_filter_values)

In [ ]:
cropped_wl, cropped_irrad, filter_wl, filter_trans = filter_wavelength_range(wl, irrad, filter_wl, filter_trans, min_wl, max_wl)

In [ ]:
len(cropped_wl), len(cropped_irrad), len(filter_wl), len(filter_trans)

In [ ]:
# Interpolate filter trans on cropped_wl
def interpolate_transmission(spectrum_wavelengths, filter_wavelengths, filter_transmission, method='linear'):
  interp_func = interp.interp1d(filter_wavelengths, filter_transmission, kind=method, bounds_error=False, fill_value=0)
  interpolated_transmission = interp_func(spectrum_wavelengths)
  return interpolated_transmission

In [ ]:
interpolated_transmission = interpolate_transmission(cropped_wl, filter_wl, filter_trans)

In [ ]:
# Plot the interpolated_transmission
plt.figure(figsize=(10, 6))
plt.plot(cropped_wl, interpolated_transmission, linestyle='-', color='b')
plt.xlabel('Wavelength [microns]')
plt.ylabel('Transmission')
plt.title('g high Filter Interpolated Transmission')
plt.grid(True)
plt.show()

Now, the easiest thing to do is to multiply our irradiance by the transmission to get the flux AFTER the filter. Remember these are arrays and python does element-wise multiplication by default.

In [ ]:
filtered_spec = $your_code_here$

Now, plot the filtered spectrum against the original spectrum. What do you notice?

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(cropped_wl, filtered_spec, ls='-', c='b', label='filtered')
plt.plot(wl, irrad, ls = '-', c='r', label='original')
plt.xlim(min_wl, max_wl)
plt.xlabel('Wavelength [microns]')
plt.ylabel('Irradiance (W/m^2/micron)')
plt.title('Solar Spectrum after g HIGH Filter')
plt.grid(True)
plt.legend()
plt.show()

Typical spectrograph+telecope optics efficiencies are about 50%. Scale the filtered result to obtain a spectrum after the optical train. Use the code to plot all three curves and comment on how this might relate to why we want larger telescopes.

In [ ]:
final_spec = $your_code_here$

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(cropped_wl, final_spec, ls='-', c='m', label='final')
plt.plot(cropped_wl, filtered_spec, ls='-', c='b', label='filtered')
plt.plot(wl, irrad, ls = '-', c='r', label='original')
plt.xlim(min_wl, max_wl)
plt.xlabel('Wavelength [microns]')
plt.ylabel('Irradiance (W/m^2/micron)')
plt.title('Solar Spectrum after g HIGH Filter')
plt.grid(True)
plt.legend()
plt.show()

*your answer here*